# Simple RNN

In [ ]:
import numpy as np

In [ ]:
class SimpleRNN:
  def __init__(self, input_size, hidden_size, output_size):
     self.hidden_size = hidden_size
     self.Wx = np.random.randn(input_size, hidden_size)
     self.Wh = np.random.randn(hidden_size, hidden_size)
     self.Wy = np.random.randn(hidden_size, output_size)
     self.bh = np.zeros((1, hidden_size))
     self.by = np.zeros((1, output_size))

  def forward(self, inputs):
    h = np.zeros((inputs.shape[0], self.hidden_size))
    outputs = []

    for t in range(inputs.shape[1]):
      h = np.tanh(np.dot(inputs[:, t, :], self.Wx) + np.dot(h, self.Wh) + self.bh)
      y = np.dot(h, self.Wy) + self.by
      outputs.append(y)
    return np.array(outputs), h

inputs = np.random.randn(5, 10, 8)
rnn = SimpleRNN(input_size=8, hidden_size=16, output_size=4)

outputs, h = rnn.forward(inputs)

print("Outputs shape:", outputs.shape)
print("Final hidden state:", h)

Outputs shape: (10, 5, 4)
Final hidden state: [[ 0.99992061 -0.53241266  0.97776873  0.99625996  0.99749836 -0.9865753
   0.99999736 -0.99999083 -0.84912118 -0.98479046  0.99989618 -0.55951793
   0.97921091 -0.93687535  0.03623493 -0.99940157]
 [-0.97750003 -0.99964098  0.99829995 -0.99644913  0.99999717 -0.99999838
  -0.96556781 -0.62797337 -0.99951779  0.75212183 -0.46313882 -0.99999998
   0.81389376  0.99999975 -0.58907662  0.98217242]
 [-0.99920993 -0.99992106  0.6041345   0.99935821 -0.99923363  1.
   0.99681646  0.99998412  1.         -0.99897739  0.99998622  0.99998329
   0.95285693  0.01964777 -0.97059064  0.30119457]
 [-0.9999082   1.         -0.99687831  0.72509085 -0.84332214 -0.98789455
  -0.99999869 -0.7656342  -1.          0.68401024  0.05019231  0.40167074
  -0.99996899  0.99870984 -0.98768952  0.98429731]
 [ 0.99998125 -0.9999371   0.91446432  0.99957131  0.99999915 -0.99904088
   0.99998317 -0.86649913 -0.98816753 -0.20944004  0.99699267 -1.
   0.80935848  0.99773768 -

Hands-on


# Hands-on

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.models import Sequential
import numpy as np

In [ ]:
url = 'https://www.gutenberg.org/files/100/100-0.txt'
text_path = tf.keras.utils.get_file('shakespeare.txt', url)

5618815/5618815 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
text = open(text_path, 'rb').read().decode(encoding='utf-8')

In [ ]:
vocab = sorted(set(text))
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

In [ ]:
text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
seq_length = 100
examples_per_epoch = len(text) // seq_length

In [ ]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [ ]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=seq_length),
    SimpleRNN(rnn_units, return_sequences=True),
    Dense(vocab_size)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 1574s 2s/step - loss: 4.8631
Epoch 2/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 1562s 2s/step - loss: 4.5012
Epoch 3/10
254/859 ━━━━━━━━━━━━━━━━━━━━ 18:22 2s/step - loss: 4.7629

In [ ]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = [char2idx[s] for s in start_string]
  text_generated = []

  temperature = 1.0

  model.reset_states()
      for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

print(generate_text(model, start_string="ROMEO: "))